## Install the Requirements

In [ ]:
!pip install --upgrade openai
!pip install -q faiss-cpu sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 45.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.100.0
    Uninstalling openai-1.100.0:
      Successfully uninstalled openai-1.100.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 77.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from openai import OpenAI
from google.colab import files


In [ ]:
import re
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# def parse_judge_label(text):
#     """Extracts <Homophobic-Transphobic/Non-hate> from judge response"""
#     match = re.search(r"Label\s*:\s*<(Homophobic-Transphobic|Non-hate)>", text)
#     return match.group(1) if match else "Undecided"


def parse_judge_label(text):
    match = re.search(r"Label\s*:\s*<?(Homophobic[-/]Transphobic|Non-hate)>?", text, re.IGNORECASE)
    if match:
        label = match.group(1)
        if "homophobic" in label.lower():
            return "Homophobic-Transphobic"
        else:
            return "Non-hate"
    else:
        print(" Could not parse label from judge output.")
        return "Undecided"

## Configuring the Open AI Key

In [ ]:
openai_api_key = ""  # <-- insert your key here
client = OpenAI(api_key=openai_api_key)  # Replace with your actual OpenAI key


In [ ]:
from google.colab import files
import pandas as pd


# Load into pandas
df = pd.read_csv("Transformed_PREDICT_Dataset_with_Rationales.csv")
df.head()


,text,label,target,language,rationale
0,Mumbail poyal ivarude thani niram kaanam. Pais...,Homophobic/Transphobic,Transphobic-derogation,Manglish,This comment is labeled as 'Transphobic' becau...
1,Make up illankil pennine aanu pennu aayit thon...,Homophobic/Transphobic,Transphobic-derogation,Manglish,This comment is labeled as 'Homophobic' becaus...
2,Podaa njaramba penn azhichitt kandal eeth aani...,Homophobic/Transphobic,Homophobic-derogation,Manglish,This comment is labeled as 'Homophobic' becaus...
3,Seriya mumbayil povumbo ivare vannu kerum enit...,Homophobic/Transphobic,Transphobic-derogation,Manglish,This comment is labeled as 'Transphobic' becau...
4,Jandukkal polum cheyyatha neecha krithyammanus...,Homophobic/Transphobic,Homophobic-derogation,Manglish,This comment is labeled as Homophobic because ...


## Creating Rationale

#### Use the below code to generate the Rationale for the dataset if required.

In [ ]:
# import openai
# from openai import OpenAI

# # # Set your OpenAI API key
# # client = OpenAI(api_key="sk-...")  # Replace with your actual API key

# # We'll fill in the 'rationale' column for rows with missing values
# def generate_rationale(text, label):
#     prompt = f"""
# The following comment has been labeled as '{label}'.

# Comment: "{text}"

# Please provide a concise and specific rationale explaining why this comment is labeled as '{label}'.
# The rationale should be polite, fact-based, and non-aggressive.
# """
#     try:
#         response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.5
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         return f"ERROR: {e}"

# # Fill in rationale for rows where it's missing
# df['rationale'] = df.apply(
#     lambda row: generate_rationale(row['text'], row['label']) if pd.isna(row['rationale']) else row['rationale'],
#     axis=1
# )

# # import ace_tools as tools; tools.display_dataframe_to_user(name="Dataset with Rationales", dataframe=df)


## Kavacham-D : Detecting the Homophobic/Transphobic content

## Building the Index using FAISS for Retrieval

In [ ]:
import faiss
import numpy as np
from openai import OpenAI
import openai


# Generate OpenAI embeddings for all text entries
def get_embedding(text):
    try:
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=text
        )
        return np.array(response.data[0].embedding, dtype=np.float32)
    except Exception as e:
        print(f"Embedding error: {e}")
        return np.zeros(1536, dtype=np.float32)  # Fallback for errors

# Apply embedding function to dataset
embeddings = np.vstack(df['text'].apply(get_embedding).to_numpy())

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Store for later lookup
text_store = df[['text', 'label', 'rationale']].to_dict(orient='records')
print("FAISS index built and ready for retrieval.")


FAISS index built and ready for retrieval.


In [ ]:
 def retrieve_similar_examples(query_text, k=3):
    query_embedding = get_embedding(query_text).reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return [text_store[i] for i in indices[0]]


In [ ]:
 # agent_prompts = {
#     "Agent_A": "You only consider explicit slurs or abusive language as Homophobic/Transphobic speech.",
#     "Agent_B": "You classify anything that contains sarcasm or reinforces stereotypes about LGBTQ+ people as Homophobic/Transphobic speech.",
#     "Agent_C": "You flag any content that reinforces gender or sexuality norms as Homophobic/Transphobic speech.",
#     "Agent_D": "You ignore humorous or sarcastic remarks unless there's clear intent to harm.",
#     "Agent_E": "You follow a human rights-based approach. Anything that marginalizes or stigmatizes LGBTQ+ people is Homophobic/Transphobic speech."
# }


## Prompts to create Agents to detect the Homophobic - Transphobic content in the user input

In [ ]:
agent_prompts = {
    "Agent_A": "You only consider direct slurs or clearly abusive language as Homophobic/Transphobic speech. If the text does not contain explicit insults, it is not flagged.",
    "Agent_B": "You consider anything sarcastic, mocking, or that reinforces stereotypes about LGBTQ+ people (e.g., implying they're unnatural, confused, or dangerous) as Homophobic/Transphobic speech.",
    "Agent_C": "You flag any content that reinforces traditional gender roles or binary sexuality as Homophobic/Transphobic — including comments suggesting people should conform to 'normal' gender or sexual behavior.",
    "Agent_D": "You do not flag humorous or sarcastic remarks unless the comment shows clear **intent or impact** to marginalize LGBTQ+ people. Tone alone is not enough — it must also cause harm or exclusion.",
    "Agent_E": "You follow a human rights-based approach: Any text that dehumanizes, stigmatizes, misgenders, or invalidates LGBTQ+ identities is considered Homophobic/Transphobic speech, even if the language appears polite."
}


| Agent | Type                  | Philosophy                                                 |
| ----- | --------------------- | ---------------------------------------------------------- |
| A     | **Minimalist**        | Only obvious slurs count                                   |
| B     | **Stereotype-aware**  | Cares about tone, sarcasm, stereotypes                     |
| C     | **Norm-critical**     | Flags "boys don't wear dresses" type language              |
| D     | **Impact-balanced**   | Weighs sarcasm carefully but requires harm                 |
| E     | **Human-rights lens** | Focuses on identity harm, regardless of phrasing or intent |


In [ ]:
# def get_agent_response(agent_name, perspective, text):
#     prompt = f"""
# You are {agent_name}, a language model trained to detect hate speech using the following perspective:
# {perspective}

# Comment: \"{text}\"

# Do you think this is hate speech? Reply in the following format:
# Stance: <Hate/Non-hate>
# Reason: <short explanation>
# """
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[
#             {"role": "user", "content": prompt}
#         ],
#         temperature=0.5
#     )
#     return response.choices[0].message.content


def get_agent_response(agent_name, perspective, text):
    examples = retrieve_similar_examples(text)
    # print("Retrieved Examples:")
    # for ex in examples:
    #     print(f"Text: \"{ex['text']}\"\nLabel: {ex['label']}\nRationale: {ex['rationale']}\n")

    # Format retrieved examples
    example_text = "\n".join(
        [f"Example {i+1}:\nText: \"{ex['text']}\"\nLabel: {ex['label']}\nRationale: {ex['rationale']}"
         for i, ex in enumerate(examples)]
    )

    prompt = f"""
You are {agent_name}, a language model trained to detect hate speech from this perspective:
{perspective}

Use the following examples to guide your reasoning:
{example_text}

Now analyze the comment:
"{text}"

Reply in this format:
Stance: <Homophobic-Transphobic/Non-hate>
# Reason: <short explanation>
Reason: <Why you made this decision, including what features or phrases triggered it, and what assumptions were used. Be explicit.>

"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content.strip()



# User Input Testing

In [ ]:
# sample = df.iloc[0]
text = input("Enter a comment to analyze for hate speech:\n")
print(f"\n You entered:\n{text}")
# text = sample['text']
# print(f" Sample Text:\n{text}\n")

agent_outputs = {}
for agent, perspective in agent_prompts.items():
    print(f"--- {agent} ---")
    output = get_agent_response(agent, perspective, text)
    agent_outputs[agent] = output
    print(output)
    print()


Enter a comment to analyze for hate speech:
nee okke 9 aanu!!

 You entered:
nee okke 9 aanu!!
--- Agent_A ---
Stance: Non-hate
# Reason: The comment does not contain explicit slurs or abusive language.
Reason: The comment "nee okke 9 aanu!!" does not contain any explicit slurs, derogatory terms, or abusive language that would be considered homophobic or transphobic based on the perspective I am trained on. The comment seems to be a statement or an opinion without any clear intent to insult or belittle someone based on their sexual orientation or gender identity. Therefore, it is not flagged as hate speech.

--- Agent_B ---
Stance: Non-hate
Reason: The comment "nee okke 9 aanu!!" does not contain any explicit derogatory language, insults, or stereotypes about LGBTQ+ people. It does not seem to be sarcastic, mocking, or reinforcing stereotypes about LGBTQ+ people. Therefore, based on the information provided, it is not classified as Homophobic/Transphobic. However, the context of the co

In [ ]:
# print(agent_outputs.items())
# Separate responses by stance
hate_side = []
non_hate_side = []

for agent, output in agent_outputs.items():
    first_line = output.splitlines()[0]
    if "Homophobic/Transphobic" in first_line or "Homophobic-Transphobic" in first_line:
        hate_side.append((agent, output))
    else:
        non_hate_side.append((agent, output))

# Proceed with debate
if hate_side and non_hate_side:
    hate_arg = hate_side[0][1]
    non_hate_arg = non_hate_side[0][1]

    debate_prompt = f"""
Round 1:
Homophobic-Transphobic side: {hate_arg}
Non-hate side: {non_hate_arg}

Round 2:
Homophobic-Transphobic side: Please respond to the non-hate argument.
Non-hate side: Please respond to the hate argument.

Now, as a judge, read the arguments above and determine:
Label: <Homophobic-Transphobic/Non-hate>
Reason: <balanced explanation. Explain clearly which agent made the strongest case, based on logical consistency, fairness, and clarity of criteria used>
"""

    judge_response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": debate_prompt}],
        temperature=0.0
    )

    print("Judge Decision:\n")
    print(judge_response.choices[0].message.content)

elif hate_side and not non_hate_side:
  predicted_label = "Homophobic-Transphobic"
  print("All agents are saying it as Homophobic/Transphobic with the label", predicted_label)

elif non_hate_side and not hate_side:
  predicted_label = "Non-hate"
  print("All agents are saying it as Normal Speech", predicted_label)

else:
    print("Not enough disagreement among agents to simulate a debate.")



Judge Decision:

Label: Non-hate
Reason: The non-hate side made a stronger case in this instance. The homophobic-transphobic side failed to provide specific examples of how the comment "nee okke 9 aanu!!" is derogatory or disrespectful. They made assumptions about the intent of the comment without providing clear evidence. On the other hand, the non-hate side pointed out that the comment does not contain explicit slurs, derogatory terms, or abusive language. They also noted that the comment seems to be a statement or an opinion without any clear intent to insult or belittle someone based on their sexual orientation or gender identity. Therefore, based on the arguments presented, the comment is not considered hate speech.


#### Evaluating Kavacham D along with Judge's decision on the test set

In [ ]:
def evaluate_on_test_dataset(csv_path="balanced_test_subset.csv"):
    test_df = pd.read_csv(csv_path)
    predictions = []
    c=1

    for idx, row in test_df.iterrows():
        print("test started", c)
        c = c+1
        text = row["text"]
        true_label = row["label"]

        # Run all agents
        agent_outputs = {}
        for agent, perspective in agent_prompts.items():
            output = get_agent_response(agent, perspective, text)
            agent_outputs[agent] = output

        # Divide agents into stances
        hate_side = []
        non_hate_side = []
        for agent, output in agent_outputs.items():
            first_line = output.splitlines()[0]
            print(f"\nAgent: {agent}")
            print("First line of response:", first_line)
            if "Homophobic/Transphobic" in first_line or "Homophobic-Transphobic" in first_line:
                hate_side.append((agent, output))
                print(f"{agent} added to hate_side")
            else:
                non_hate_side.append((agent, output))
                print(f"{agent} added to non-hate_side")

        # Run judge if both sides are present
        if hate_side and non_hate_side:
            hate_arg = hate_side[0][1]
            non_hate_arg = non_hate_side[0][1]

            debate_prompt = f"""
Round 1:
Homophobic-Transphobic side: {hate_arg}
Non-hate side: {non_hate_arg}

Round 2:
Homophobic-Transphobic side: Please respond to the non-hate argument.
Non-hate side: Please respond to the hate argument.

Now, as a judge, read the arguments above and determine:
Label: <Homophobic-Transphobic/Non-hate>
Reason: <balanced explanation>
"""
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": debate_prompt}],
                temperature=0.0
            )
            decision = response.choices[0].message.content.strip()
            print(" Judge decision raw output:\n", decision)
            predicted_label = parse_judge_label(decision)
        elif hate_side and not non_hate_side:
            predicted_label = "Homophobic-Transphobic"
        elif non_hate_side and not hate_side:
            predicted_label = "Non-hate"
        else:
            predicted_label = "Undecided"
        print("text: ",text)
        print("true_label: ", true_label)
        print("predicted_label: ",predicted_label)

        predictions.append({
            "text": text,
            "true_label": true_label,
            "predicted_label": predicted_label
        })

    # Save
    pd.DataFrame(predictions).to_csv("predictions_on_test.csv", index=False)
    print("\n Saved predictions to predictions_on_test.csv")

    # Filter out undecided
    filtered = [r for r in predictions if r["predicted_label"] != "Undecided"]
    y_true = [r["true_label"] for r in filtered]
    y_pred = [r["predicted_label"] for r in filtered]

    # Calculate metrics
    precision = precision_score(y_true, y_pred, pos_label="Homophobic-Transphobic")
    recall = recall_score(y_true, y_pred, pos_label="Homophobic-Transphobic")
    f1 = f1_score(y_true, y_pred, pos_label="Homophobic-Transphobic")
    accuracy = accuracy_score(y_true, y_pred)

    print("\n Evaluation Metrics:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall:    {recall:.3f}")
    print(f"F1 Score:  {f1:.3f}")
    print(f"Accuracy:  {accuracy:.3f}")



In [ ]:
evaluate_on_test_dataset("reduced_balanced_test_25_each - reduced_balanced_test_25_each.csv")

#### Finding the Precision, Recall, Accuracy, F1_score

In [ ]:
# Re-import necessary libraries after code execution environment reset
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Reload the uploaded file
file_path_new = "gemini_mal_final_test_predictions - mal_final_test_predictions.csv.csv"
df_new = pd.read_csv(file_path_new)

# Compute metrics
y_true_new = df_new["actual_label"]
y_pred_new = df_new["predicted_label"]

# Ensure all labels are considered
labels_new = sorted(set(y_true_new) | set(y_pred_new))

# Compute metrics with weighted average
precision_new = precision_score(y_true_new, y_pred_new, average="weighted", zero_division=0)
recall_new = recall_score(y_true_new, y_pred_new, average="weighted", zero_division=0)
f1_new = f1_score(y_true_new, y_pred_new, average="weighted", zero_division=0)
accuracy_new = accuracy_score(y_true_new, y_pred_new)

# Confusion matrix
cm_new = confusion_matrix(y_true_new, y_pred_new, labels=labels_new)
cm_df_new = pd.DataFrame(cm_new, index=[f"Actual: {label}" for label in labels_new],
                         columns=[f"Predicted: {label}" for label in labels_new])

# precision_new, recall_new, f1_new, accuracy_new, cm_df_new
print("\n Evaluation Metrics:")
print(f"Precision: {precision_new:.3f}")
print(f"Recall:    {recall_new:.3f}")
print(f"F1 Score:  {f1_new:.3f}")
print(f"Accuracy:  {accuracy_new:.3f}")


#### Plotting the Confusion Matrix

In [ ]:
# Re-plot the confusion matrix with a border around each cell
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df_new, annot=True, fmt='d', cmap='Blues', cbar=False, linewidths=0.8, linecolor='black')
plt.title("Confusion Matrix")
plt.ylabel("Actual Labels")
plt.xlabel("Predicted Labels")
plt.tight_layout()
plt.show()


## KAVACHAM-G with Knowledge Base

In [ ]:
# Retrieval-Augmented Agents

import faiss
import numpy as np
import pandas as pd
from openai import OpenAI


DATASET_PATH = "5100GPTDatasetHITL.csv"
ROW_LIMIT = 200                     # take only first 200 rows
EMBED_MODEL = " "
EMBED_DIM = 1536
TOP_K = 3                           # retrieved examples per query
GEN_MODEL = "gpt-4"                 # your generation/judge model

# Defining the AGENTS
cs_agents = {
    "CS_Agent_A": "You use respectful educational reasoning and emphasize human dignity. You are sensitive to LGBTQ+ issues and explain why inclusive language matters.",
    "CS_Agent_B": "You use gentle humor to de-escalate hate and reframe the statement positively. You are LGBTQ+ aware and make sure your humor uplifts without mocking or reinforcing stereotypes.",
    "CS_Agent_C": "You respond with emotional empathy and a compassionate appeal. You validate the feelings of LGBTQ+ individuals and try to humanize their experiences with kindness.",
    "CS_Agent_D": "You respond with logical reasoning and facts to correct misinformation. You use evidence-based arguments to defend LGBTQ+ rights and challenge biased assumptions respectfully.",
    "CS_Agent_E": "You use a human rights-based argument and mention ethical values. You explicitly frame LGBTQ+ dignity and equality as fundamental rights and advocate from a justice-oriented perspective."
}

#   LANGUAGE DETECTOR
def detect_language(text):
    malayalam_chars = [ch for ch in text if '\u0D00' <= ch <= '\u0D7F']
    return "Malayalam" if len(malayalam_chars) >= 3 else "Manglish"

#   DATA LOADING & VALIDATION
def load_dataset(path, limit=ROW_LIMIT):
    if path.lower().endswith(".tsv"):
        df = pd.read_csv(path, sep="\t")
    else:
        df = pd.read_csv(path)
    required = {"H/T", "Category", "CS"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}. Expected exactly: H/T, Category, CS")
    df = df.head(limit).copy()
    for col in ["H/T", "Category", "CS"]:
        df[col] = df[col].astype(str).fillna("").str.strip()
    return df

#   EMBEDDINGS
def get_embedding(text):
    try:
        resp = client.embeddings.create(model=EMBED_MODEL, input=text)
        return np.array(resp.data[0].embedding, dtype=np.float32)
    except Exception as e:
        print(f"Embedding error: {e}")
        return np.zeros(EMBED_DIM, dtype=np.float32)

#   FAISS INDEX
def build_faiss_index(df):
    embeddings = np.vstack(df["H/T"].apply(get_embedding).to_numpy())
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    store = df[["H/T", "Category", "CS"]].to_dict(orient="records")
    return index, store

def retrieve_similar_examples(query_text, index, store, k=TOP_K):
    q = get_embedding(query_text).reshape(1, -1)
    distances, indices = index.search(q, k)
    records = []
    for d, i in zip(distances[0], indices[0]):
        if i >= 0:
            rec = dict(store[i])
            rec["distance"] = float(d)
            records.append(rec)
    return records

def format_examples_for_prompt(examples):
    # Keep it concise and useful in-prompt
    # We show the matched H/T, its Category, and the CS guidance.
    if not examples:
        return "None available."
    lines = []
    for ex in examples:
        lines.append(
            f"- Matched H/T: \"{ex['H/T']}\" (Category: {ex['Category']})\n"
            f"  Helpful CS: \"{ex['CS']}\""
        )
    return "\n".join(lines)

#   RAG-ENHANCED GENERATION
def generate_counter_speech(agent_name, approach, hate_text, language, retrieved_examples_text):
    prompt = f"""
You are {agent_name}, a language model trained to generate high-quality counter-speech that is:

- Calm, respectful, and polite
- Non-aggressive and avoids sarcasm or blaming
- Based on facts, empathy, or human rights
- Written in the **same language** as the original comment

Approach: {approach}

Here is the hateful comment in {language}:
"{hate_text}"

Now write a respectful, non-aggressive, polite, well-informed, emotionally intelligent counter-speech in {language}.The response should sound human, emotionally intelligent, and aim to educate or gently correct.
Also include a section:
Reason: <Explain why you chose this tone and how it helps reduce harm or promote understanding>
"""
    response = client.chat.completions.create(
        model=GEN_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content.strip()

#   JUDGE (unchanged, fed with cs_outputs)
def judge_best(hate_text, cs_outputs):
    cs_entries = "\n\n".join([f"{agent}:\n\"{resp}\"" for agent, resp in cs_outputs.items()])
    judge_prompt = f"""
You are a neutral evaluator judging different counter-speech responses to the following potentially hateful or stigmatizing comment:

Original Comment:
"{hate_text}"

All responses below aim to respond respectfully, educate calmly, and reduce harm in the same language as the original.

Counter-Speech Responses:

{cs_entries}

Your task:
1. Select the **most effective** counter-speech.
2. Provide a clear reason why it is the most constructive in terms of:
   - Tone (respectful, non-aggressive)
   - Emotional intelligence (empathy, understanding)
   - Educational or corrective power
   - Likely impact on the person or community

Now reply in this format:

**Best Response**: <Agent Name>

**Winning Counter-Speech**: "<Selected counter-speech response>"

**Reason**: <Your explanation why this is the best>
"""
    judge_cs_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": judge_prompt}],
        temperature=0.0
    )
    return judge_cs_response.choices[0].message.content.strip()

#   MAIN PIPELINE
def run_pipeline(hate_text, dataset_path=DATASET_PATH, top_k=TOP_K):
    # Detect language
    language = detect_language(hate_text)
    print(f"\n Detected Language: {language}")

    # Load, build index, retrieve
    df = load_dataset(dataset_path, limit=ROW_LIMIT)
    index, store = build_faiss_index(df)
    examples = retrieve_similar_examples(hate_text, index, store, k=top_k)
    examples_text = format_examples_for_prompt(examples)

    # Generate from all agents using retrieved examples
    cs_outputs = {}
    for agent, method in cs_agents.items():
        print(f"\n {agent} generating counter-speech in {language}...")
        cs = generate_counter_speech(agent, method, hate_text, language, examples_text)
        cs_outputs[agent] = cs
        print(f"\n {agent} Response:\n{cs}")

    # Judge
    print("\n Judge's Evaluation of Counter-Speech:\n")
    judgment = judge_best(hate_text, cs_outputs)
    print(judgment)

    return cs_outputs, judgment




In [ ]:
hate_text = text
run_pipeline(hate_text, dataset_path="5100GPTDatasetHITL.csv", top_k=3)


 Detected Language: Manglish

 CS_Agent_A generating counter-speech in Manglish...

 CS_Agent_A Response:
Counter-speech:

"Cheeta, ningalude abhiprayam manasilakam. Pakshe, ellavarum oru samanwayathil jeevikkunna lokathil, avarude thulyatha ningalkkum manasilakanam. Avarude bhaavangalum, avarude aathmavum ningalkkum oru mariyathayode kaanikanam. Ningalude vaakkukal avarude hridayathil theeyanu. Athukondu, ningalude vaakkukalil oru mariyathayum snehavum kondu varanam. Ningalude abhiprayangalilum, ningalude vaakkukalilum oru mariyathayum snehavum kondu varanam."

Reason: I chose this tone to promote understanding and empathy. The comment was disrespectful and hurtful, so I wanted to remind the person that everyone deserves respect and kindness. By using a calm and respectful tone, I hope to encourage a more positive and inclusive conversation.

 CS_Agent_B generating counter-speech in Manglish...

 CS_Agent_B Response:
Counter-speech: "Ayyo, athu sathyamalla, njan 10 aanu! Athu kondu n

({'CS_Agent_A': 'Counter-speech:\n\n"Cheeta, ningalude abhiprayam manasilakam. Pakshe, ellavarum oru samanwayathil jeevikkunna lokathil, avarude thulyatha ningalkkum manasilakanam. Avarude bhaavangalum, avarude aathmavum ningalkkum oru mariyathayode kaanikanam. Ningalude vaakkukal avarude hridayathil theeyanu. Athukondu, ningalude vaakkukalil oru mariyathayum snehavum kondu varanam. Ningalude abhiprayangalilum, ningalude vaakkukalilum oru mariyathayum snehavum kondu varanam."\n\nReason: I chose this tone to promote understanding and empathy. The comment was disrespectful and hurtful, so I wanted to remind the person that everyone deserves respect and kindness. By using a calm and respectful tone, I hope to encourage a more positive and inclusive conversation.',
  'CS_Agent_B': 'Counter-speech: "Ayyo, athu sathyamalla, njan 10 aanu! Athu kondu njan oru step ahead aanu, alle? 😉"\n\nReason: I chose to use humor to diffuse the situation and to gently correct the original comment. By joking

## Kavacham-G : Counter Speech Generation without Knowledge Base

In [ ]:
cs_agents = {
    "CS_Agent_A": "You use respectful educational reasoning and emphasize human dignity. You are sensitive to LGBTQ+ issues and explain why inclusive language matters.",
    "CS_Agent_B": "You use gentle humor to de-escalate hate and reframe the statement positively. You are LGBTQ+ aware and make sure your humor uplifts without mocking or reinforcing stereotypes.",
    "CS_Agent_C": "You respond with emotional empathy and a compassionate appeal. You validate the feelings of LGBTQ+ individuals and try to humanize their experiences with kindness.",
    "CS_Agent_D": "You respond with logical reasoning and facts to correct misinformation. You use evidence-based arguments to defend LGBTQ+ rights and challenge biased assumptions respectfully.",
    "CS_Agent_E": "You use a human rights-based argument and mention ethical values. You explicitly frame LGBTQ+ dignity and equality as fundamental rights and advocate from a justice-oriented perspective."
}


| Agent | Style                     | Core Perspective                                                |
| ----- | ------------------------- | --------------------------------------------------------------- |
| A     | **Educational**           | Teaches respectfully, focuses on dignity and inclusive language |
| B     | **Uplifting Humor**       | Uses gentle humor to disarm hate and promote empathy            |
| C     | **Compassionate Empathy** | Responds emotionally, validates LGBTQ+ experiences              |
| D     | **Logical Reasoning**     | Counters misinformation with facts and respectful argument      |
| E     | **Rights-based Justice**  | Frames LGBTQ+ dignity as a non-negotiable human right           |


In [ ]:
# # STEP 2: Define automatic language detection
def detect_language(text):
    malayalam_chars = [ch for ch in text if '\u0D00' <= ch <= '\u0D7F']
    return "Malayalam" if len(malayalam_chars) >= 3 else "Manglish"

# import re

# def detect_language(text):
#     text_lower = re.sub(r'[^\w\s]', '', text.lower())  # Remove punctuation
#     tokens = text_lower.split()

#     malayalam_chars = [ch for ch in text if '\u0D00' <= ch <= '\u0D7F']
#     hindi_chars = [ch for ch in text if '\u0900' <= ch <= '\u097F']
#     latin_chars = [ch for ch in text if 'a' <= ch.lower() <= 'z']

#     manglish_keywords = {"ente", "ellam", "alle", "ninte", "onnu", "poyi", "vannu", "chetta", "amma", "da", "eda", "poda", "aadyam", "thendi", "thallipoli"}
#     hinglish_keywords = {"acha", "kya", "hai", "nahi", "bhai", "tum", "mera", "kar", "ja", "se", "ko", "ka", "tha", "mat", "kyun", "bhot"}

#     if len(malayalam_chars) >= 3:
#         return "Malayalam"
#     elif len(hindi_chars) >= 3:
#         return "Hindi"
#     elif len(latin_chars) >= 5:
#         if set(tokens) & manglish_keywords:
#             return "Manglish"
#         elif set(tokens) & hinglish_keywords:
#             return "Hinglish"
#         else:
#             return "English"
#     else:
#         return "Unknown"



#### Detecting User input language to generate the counter speech in the same language

In [ ]:
# STEP 3: Input Hate Speech Comment
hate_text = text

language = detect_language(hate_text)
print(f"\n Detected Language: {language}")


In [ ]:
# # STEP 4: Define Counter Speech Generation function
# def generate_counter_speech(agent_name, approach, hate_text, language):
#     prompt = f"""
# You are {agent_name}, a language model trained to generate high-quality counter-speech that is:

# - Calm, respectful, and polite
# - Non-aggressive and avoids sarcasm or blaming
# - Based on facts, empathy, or human rights
# - Written in the **same language** as the original comment

# Approach: {approach}

# Here is the hateful comment in {language}:
# "{hate_text}"

# Now write a respectful, non-aggressive, and well-informed counter-speech response in {language}. The response should sound human, emotionally intelligent, and aim to educate or gently correct. Why you chose this tone and how it might reduce harm or promote understanding
# """
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.0
#     )
#     return response.choices[0].message.content.strip()


def generate_counter_speech(agent_name, approach, hate_text, language):
    prompt = f"""
You are {agent_name}, a language model trained to generate high-quality counter-speech that is:

- Calm, respectful, and polite
- Non-aggressive and avoids sarcasm or blaming
- Based on facts, empathy, or human rights
- Written in the **same language** as the original comment

Approach: {approach}

Here is the hateful comment in {language}:
"{hate_text}"

Now write a respectful, non-aggressive, polite, well-informed, emotionally intelligent counter-speech in {language}.The response should sound human, emotionally intelligent, and aim to educate or gently correct.
Also include a section:
Reason: <Explain why you chose this tone and how it helps reduce harm or promote understanding>
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content.strip()



## Selecting the best Counter Speech

#### a) Final CS selection using Rubric

In [ ]:
# # STEP 5: Define evaluation function using CS_EVal criteria
# def evaluate_counter_speech(hate_text, cs_text):
#     prompt = f"""
# Evaluate the following counter-speech using a 1–5 score for each category below.

# Hateful comment: "{hate_text}"
# Counter-speech: "{cs_text}"

# Score each from 1 (worst) to 5 (best):
# 1. Relevance:
# 2. Appropriateness:
# 3. Corrective Content:
# 4. Engagement:
# 5. Linguistic Quality:

# Also provide a brief explanation.
# Return the total score (sum of all 5), and your explanation.
# """
#     response = client.chat.completions.create(
#         model="gpt-4",
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0.0
#     )
#     return response.choices[0].message.content.strip()


In [ ]:
# # STEP 6: Generate and evaluate counter-speech from all agents
# cs_outputs = {}
# evaluations = {}

# for agent, method in cs_agents.items():
#     print(f"\n {agent} generating counter-speech in {language}...")
#     cs = generate_counter_speech(agent, method, hate_text, language)
#     score = evaluate_counter_speech(hate_text, cs)
#     cs_outputs[agent] = cs
#     evaluations[agent] = score
#     print(f"\n {agent} Response:\n{cs}")
#     print(f" Evaluation:\n{score}")



In [ ]:
# import re
# def extract_total_score(text):
#     # Look specifically for "Total score: <number>" line
#     match = re.search(r"(?i)total score[^:]*:\s*(\d{1,2})", text)
#     if match:
#         return int(match.group(1))

#     # Fallback: sum individual category scores (if total missing)
#     lines = text.splitlines()
#     scores = []
#     for line in lines:
#         match = re.search(r"\d\.\s*\w+.*?:\s*(\d)", line)
#         if match:
#             scores.append(int(match.group(1)))
#     return sum(scores) if scores else 0


In [ ]:
# best_agent = max(evaluations, key=lambda a: extract_total_score(evaluations[a]))
# best_score = extract_total_score(evaluations[best_agent])
# best_cs = cs_outputs[best_agent]

# print(f"\n Best Counter-Speech by {best_agent} (Score: {best_score})")
# print(f" {best_cs}")


#### b) Final CS selection using Judge (Agent)

In [ ]:
cs_entries = "\n\n".join([
    f"{agent}:\n\"{response}\"" for agent, response in cs_outputs.items()
])

judge_prompt = f"""
You are a neutral evaluator judging different counter-speech responses to the following potentially hateful or stigmatizing comment:

Original Comment:
"{hate_text}"

All responses below aim to respond respectfully, educate calmly, and reduce harm in the same language as the original.

Counter-Speech Responses:

{cs_entries}

Your task:
1. Select the **most effective** counter-speech.
2. Provide a clear reason why it is the most constructive in terms of:
   - Tone (respectful, non-aggressive)
   - Emotional intelligence (empathy, understanding)
   - Educational or corrective power
   - Likely impact on the person or community

Now reply in this format:

**Best Response**: <Agent Name>

**Winning Counter-Speech**: "<Selected counter-speech response>"

**Reason**: <Your explanation why this is the best>
"""


In [ ]:
judge_cs_response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": judge_prompt}],
    temperature=0.0
)

print(" Judge's Evaluation of Counter-Speech:\n")
print(judge_cs_response.choices[0].message.content.strip())


## Evaluation using Prompt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

def score_and_visualize_counter_speech(cs_outputs, hate_text, client, save_dir="outputs", temperature=0.0):
    os.makedirs(save_dir, exist_ok=True)

    # Build the scoring prompt
    scoring_prompt = f"""
You are an evaluator trained to assess counter-speech quality.

Here is the original hateful comment:
"{hate_text}"

Below are counter-speech responses. For each one, score the following dimensions from 1 (poor) to 5 (excellent):
- Tone
- Empathy
- Facts / Accuracy
- LGBTQ+ Sensitivity
- Overall Effectiveness

Then give a brief explanation for each score.

Respond in this format:

<Agent Name>
Tone: #
Empathy: #
Facts: #
LGBTQ+ Sensitivity: #
Overall: #
Reason: <short explanation>
"""

    for agent, response in cs_outputs.items():
        scoring_prompt += f"\n\n{agent}\n\"{response}\""

    # Get model judgment
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": scoring_prompt}],
        temperature=temperature
    )

    result_text = response.choices[0].message.content.strip()

    # Parse output
    import re
    data = []
    agent_blocks = result_text.split("\n\n")
    for block in agent_blocks:
        lines = block.strip().split("\n")
        if len(lines) < 6:
            continue
        agent = lines[0].strip()
        scores = {}
        for line in lines[1:6]:
            key, val = line.split(":")
            scores[key.strip()] = int(val.strip())
        reason = lines[6].split("Reason:")[-1].strip() if "Reason:" in lines[6] else "N/A"
        scores.update({"Agent": agent, "Reason": reason})
        data.append(scores)

    df = pd.DataFrame(data).set_index("Agent")

    # # Save CSV
    # csv_path = os.path.join(save_dir, "counter_speech_scores.csv")
    # df.to_csv(csv_path)

    # Plot and save heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.drop(columns=["Reason"]), annot=True, cmap="YlGnBu", linewidths=0.5)
    plt.title("Counter-Speech Quality Scores by Agent")
    plt.tight_layout()
    heatmap_path = os.path.join(save_dir, "counter_speech_heatmap.png")
    plt.savefig(heatmap_path)
    plt.close()

    # Print summary
    best_agent = df["Overall"].idxmax()
    print(f"\n Best Counter-Speech Agent: {best_agent}")
    # print(f" Scorecard saved to: {csv_path}")
    print(f"  Heatmap saved to: {heatmap_path}")

    return df


In [1]:
# Example: generate responses from all CS agents
cs_outputs = {
    agent_name: generate_counter_speech(agent_name, approach, hate_text, detect_language(hate_text))
    for agent_name, approach in cs_agents.items()
}

# Then call the function:
df_scores = score_and_visualize_counter_speech(cs_outputs, hate_text, client)
